In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

####Data Reading

In [0]:
df=spark.read.format('parquet').load('abfss://bronze@databrickseteprojectsa.dfs.core.windows.net/products')
df.display()

In [0]:
df=df.drop('_rescued_data')

In [0]:
df.createOrReplaceTempView('products')

In [0]:
%sql
select * from products

####**Functions**

#####Creating function using sql

In [0]:
%sql
create or replace function databricks_catalog.bronze.discounted_price(p_price double)
returns double
language sql
return p_price*0.90

In [0]:
%sql
select product_id,price,databricks_catalog.bronze.discounted_price(price) as discounted_price from products

####Using function in dataframe directly

In [0]:
df=df.withColumn('discounted_price',expr('databricks_catalog.bronze.discounted_price(price)'))
df.display()

####Function using python

In [0]:
%sql
create or replace function databricks_catalog.bronze.products_brand_upper(p_brand string)
returns string
language python
as
$$
  return p_brand.upper()
$$

In [0]:
%sql
select *,databricks_catalog.bronze.products_brand_upper(brand) as Upper_brand
from products

####Creating products table and doing Upsert

In [0]:
from delta.tables import DeltaTable

In [0]:
#ensure one row per Product_id in the incoming batch
df_products=df.dropDuplicates(['product_id'])

if spark.catalog.tableExists("databricks_catalog.silver.products"):
    dlt_obj = DeltaTable.forName(spark, "databricks_catalog.silver.products")
    dlt_obj.alias("trg").merge(df_products.alias("src"),"trg.product_id=src.product_id").whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

else:
    (df_products.write
        .format("delta")
        .mode("overwrite")   # only for first creation
        .saveAsTable(databricks_catalog.silver.products)
    )


####Enabling changeDataFeed

In [0]:
%sql
ALTER TABLE databricks_catalog.silver.products
SET TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true');
